In [2]:
from deepface import DeepFace

2024-04-27 15:23:46.300595: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 15:23:47.131292: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
DeepFace.analyze(img_path="/home/akshilmy/MobileApps/health_companion_app/server/temp/8d272092_akshilmy_face/0113.jpg",actions=['emotion'])

2024-04-27 15:24:01.705658: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-27 15:24:01.749761: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[{'emotion': {'angry': 0.0004766885297069707,
   'disgust': 2.521355431596674e-16,
   'fear': 5.020502713044758e-06,
   'happy': 4.2696596027909145e-08,
   'sad': 0.00741579395799042,
   'surprise': 1.8283464374808676e-09,
   'neutral': 99.99210238409653},
  'dominant_emotion': 'neutral',
  'region': {'x': 176,
   'y': 708,
   'w': 697,
   'h': 697,
   'left_eye': (656, 978),
   'right_eye': (400, 995)},
  'face_confidence': 0.91}]

In [25]:
import json
import random

def load_questions_from_files(file_paths):
    questions = []
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            data = json.load(file)
            questions.extend(data['questions'])
    return questions

import random

def choose_random_questions(questions, total_questions):
    random.shuffle(questions)
    selected_questions = []
    remaining_questions = total_questions
    prev_tag = None  # To keep track of the previous tag
    while remaining_questions > 0:
        random.shuffle(questions)
        for question in questions:
            # Check if the question's tag is different from the previous one
            if prev_tag != question['tag']:
                if remaining_questions == 0:
                    break
                selected_questions.append(question)
                remaining_questions -= 1
                prev_tag = question['tag']
    return selected_questions


# Example usage
file_paths = ['/home/akshilmy/MobileApps/health_companion_app/server/app/questionnaire/image_feedback.json',
              '/home/akshilmy/MobileApps/health_companion_app/server/app/questionnaire/jokes.json',
              '/home/akshilmy/MobileApps/health_companion_app/server/app/questionnaire/mental_health.json'
             ]
total_questions = 10 # Total number of questions you want to select
questions = load_questions_from_files(file_paths)
selected_questions = choose_random_questions(questions, total_questions)

# Print the selected questions
for i, question in enumerate(selected_questions, start=1):
    print(f"{i}. {question['question']}")


1. Here's a joke. What do you call fake spaghetti?
2. How often do you feel worthless or guilty?
3. Here's a joke. Why did the tomato turn red?
4. Have you had difficulty concentrating, making decisions, or remembering things?
5. Here's a joke. How do you organize a space party?
6. How does this image make you feel?
7. Here's a joke. What did one wall say to the other wall?
8. What do you think the main subject of this image is thinking or feeling?
9. How often have you had trouble falling asleep, staying asleep, or sleeping too much?
10. Here's a joke. What did the ocean say to the beach?


In [26]:
questions

[{'id': 'joke3',
  'question': "Here's a joke. What do you call fake spaghetti?",
  'tag': 1,
  'options': {'An impasta!': True,
   'A noodle impostor.': True,
   "I don't find it funny.": False,
   "I can't stop laughing!": True}},
 {'id': 'joke10',
  'question': "Here's a joke. Why don't skeletons fight each other?",
  'tag': 1,
  'options': {"They don't have the guts!": True,
   "Because they're too fragile.": True,
   "I don't find it funny.": False,
   "I'm rolling on the floor laughing!": True}},
 {'id': 'health5',
  'question': 'How often do you feel worthless or guilty?',
  'tag': 2,
  'options': {'Not at all': True,
   'Several days': False,
   'More than half the days': False,
   'Nearly every day': False}},
 {'id': 'health0',
  'question': 'How often have you felt down, depressed, or hopeless in the past two weeks?',
  'tag': 2,
  'options': {'Not at all': True,
   'Several days': False,
   'More than half the days': False,
   'Nearly every day': False}},
 {'id': 'joke6',
  

In [33]:
answers = [
    {"id": "joke7", "response": "Why are you so sandy?"},
    {"id": "health4", "response": "Nearly every day"},
    {"id": "img0", "response": "Sad"},
    {"id": "health1", "response": "Nearly every day"},
    {"id": "joke9", "response": "Stop staring ats me."},
    {"id": "joke4", "response": "I don't find it funny."},
    {"id": "joke6", "response": "I don't find it funny."},
    {"id": "joke11", "response": "I don't find it funny."},
    {"id": "health0", "response": "Nearly every day"},
    {"id": "health6", "response": "Nearly every day"},
]


In [37]:
def evaluate_answers(questions, answers):
    results = []

    for answer in answers:
        question_id = answer['id']
        response = answer['response']
        question_found = False

        for question in questions:
            if question['id'] == question_id:
                question_found = True
                if 'options' in question:
                    options = question['options']
                    if response in options and options[response]:
                        results.append(True)
                    else:
                        results.append(False)

        if not question_found:
            results.append(False)  # If the question ID is not found, consider the answer incorrect

    return results

# Assuming 'questions' and 'answers' are the provided JSON data
results = evaluate_answers(questions, answers)
print(results)
print(results.count(False)/len(results))

[True, False, False, False, True, False, False, False, False, False]
0.8
